# Load data to Colab

In [17]:
if False: # Manual loading
    from google.colab import file
    uploaded = files.upload() # then browse, select the files. It's then uploaded 
else: # Automatic loading
    import requests
    import gzip
    
    filepath_d_gr = 'http://users.diag.uniroma1.it/challenge9/data/USA-road-d/' + 'USA-road-d.NY.gr.gz'
    filepath_t_gr = 'http://users.diag.uniroma1.it/challenge9/data/USA-road-t/' + 'USA-road-t.NY.gr.gz'
    filepath_d_co = 'http://users.diag.uniroma1.it/challenge9/data/USA-road-d/' + 'USA-road-d.NY.co.gz'
    
    def loader(url):
        name = url.rsplit('/', 1)[1].rsplit('.', 1)[0]
        savename = name + '.txt'
        
        with open(savename, 'wb') as f_out:
            with requests.get(url) as r:
                f_in = gzip.decompress(r.content)
                f_out.write(f_in)
                
        print(savename)
            
    loader(filepath_d_gr)
    loader(filepath_t_gr)
    loader(filepath_d_co)

USA-road-d.NY.gr.txt
USA-road-t.NY.gr.txt
USA-road-d.NY.co.txt


# Provided code

## Graph and Vertex classes

In [1]:
# Vertex implementation
class Vertex:
    # Initialization of a vertex, given a neighbor and the corresponding weight
    # Each vertex contains a list of neighbors and corresponding weights
    def __init__(self, i, neighbor_index, weight):
        self.index = i
        self.neighbors = [neighbor_index]
        self.weights = [weight]
        
    def get_neighbors(self):
        return self.neighbors
    
    def get_weights(self):
        return self.weights
    
    # Add a neighbor with corresponding weight to the vertex
    def add_neighbor(self, neighbor_index, weight):
        self.neighbors.append(neighbor_index)
        self.weights.append(weight)


# Graph data structure
class Graph:
    # Initializes a graph with n_vertices nodes
    # The graph contains a list of vertices
    def __init__(self, n_vertices):
        self.vertices = [None] * (n_vertices+1)
        self.num_vertices = len(self.vertices)
    
    # Returns the i'th node
    def get_vertex(self, i):
        if (i > len(self.vertices)) | (i <= 0):
            raise ValueError(f'index {i} is out of bounds')
        else:
            return self.vertices[i]
    
    # Adds a new vertex to the graph
    def add_vertex(self, vertex_index, neighor_index, distance):
        if self.vertices[vertex_index] is None:
            # Construct new vertex
            self.vertices[vertex_index] = Vertex(vertex_index, neighor_index, distance)
        else:
            # Vertex already in graph but other neighbor, add extra edge
            self.vertices[vertex_index].add_neighbor(neighor_index, distance)




In [2]:
import fileinput

# Read graph data
def read_graph(file_path):
    num_vertices = 0
    for line in fileinput.input([file_path]):
        words = line.split(" ")
        if words[0] == "p":
            num_vertices = int(words[2])
    graph = Graph(num_vertices)
    for line in fileinput.input([file_path]):
        words = line.split(" ")
        if words[0] == "a":
            graph.add_vertex(int(words[1]), int(words[2]), float(words[3]))
    return graph


# read coordinates data
def read_coordinates(filepath):
    # Start to count from 1
    coordinates = [None]
    for line in fileinput.input([filepath]):
        words = line.split(" ")
        if words[0] == "v":
            coordinates.append([float(words[2]), float(words[3])])
    return coordinates


## Usefull functions

In [3]:
import numpy as np
    
# Priority queue definition
class PriorityQueue(dict):
    def put(self, item, value):
        # Watch out that value is not overwritten with higher value, shouldn't be allowed to happen!
        self[item] = value  
    
    def pop(self):
        """
        Returns the item with the lowest weight
        """
        item_min = min(self, key=self.get)
        super().pop(item_min)
        return item_min

    
def angles2centimeters(lo, la):
    """
    Convert longitude and latitude to local orthogonal grid
    :param lo: longitude
    :param la: latitude
    :return: height and width coordinates in cm's
    """
    radius = 6300 * 1e4  # cm
    la_mean = 40794234.  # 1e-6 degree
    lo_mean = -74016939.  # 1e-6 degree
    
    w = radius * np.cos(np.radians(la / 1e6)) * np.radians((lo - lo_mean) / 1e6)
    h = radius * np.radians((la - la_mean) / 1e6)
    
    return w, h 

# Assignment

## Read the data

In [7]:
# Read graph data and coordinates data (to be implemented)
graph = read_graph('USA-road-d.NY.gr.txt')
co = read_coordinates('USA-road-d.NY.co.txt')

print(f'Vertices: {graph.num_vertices-1}')
num_edges = 0
for i in range(1, graph.num_vertices):
    num_edges += len(graph.get_vertex(i).get_neighbors())
print(f'Edges: {num_edges}')

Vertices: 264346
Edges: 733846


## A* implementation

### Heuristic

In [17]:
# TO BE IMPLEMENTED
import math

def h(node1, node2):
    """
    Heuristic function
    """
    co1 = co[node1]
    x1, y1 = angles2centimeters(co1[0], co1[1])
    
    co2 = co[node2]
    x2, y2 = angles2centimeters(co2[0], co2[1])
    
    # Euclidean
    # distance = round(math.sqrt((x1-x2)**2 + (y1-y2)**2))
    
    # Manhattan
    #distance = abs(x1-x2) + abs(y1-y2)
    
    # admissible but non-consistent
    # if (node1 + node2) % 2 == 0:
    #     distance = 0
    # else:
    #     distance = round(math.sqrt((x1-x2)**2 + (y1-y2)**2))
        
    # no heuristic
    distance = 0
    
    return distance

### Algorithm

In [13]:

###########################################
########## A* implementation ##############
###########################################

# TO BE IMPLEMENTED
def a_star_search():
    """
    A* algorithm
    :param graph: Graph object
    :param co: coordinates list
    :param start: index of start node
    :param goal: index of start node
    :return: The path of nodes and total length
    """
    open = [start]
    closed = []
    came_from = {}
    g = {start: 0}
    f = PriorityQueue()
    f.put(start, h(start, goal))

    while len(f) > 0:
        current = f.pop()
        if current == goal:
            reversed_path = []
            while current != start:
                reversed_path.append(current)
                current = came_from[current]
            reversed_path.append(current)
            path = [x for x in reversed(reversed_path)]
            cost = g[goal]
            return path, cost
        
        open.remove(current)
        closed.append(current)

        for index, neighbor in enumerate(graph.get_vertex(current).get_neighbors()):
            if neighbor in closed:
                continue
            
            tentative_g_score = g[current] + graph.get_vertex(current).get_weights()[index]
            if neighbor not in open:
                open.append(neighbor)
            elif tentative_g_score >= g[neighbor]:
                continue
            
            came_from[neighbor] = current
            g[neighbor] = tentative_g_score
            f.put(neighbor, g[neighbor] + h(neighbor, goal))

In [18]:
import random

gn = 16
N = 264346
random.seed(gn)
start = random.randint(1, N+1)
goal = random.randint(1, N+1)
path, cost = a_star_search()
print(f'Path from {start} to {goal}: {path}')
print(f'Length: {len(path)-1} (excluding start)')
print(f'Cost: {cost}')

Path from 189543 to 246008: [189543, 189542, 189541, 189540, 189462, 189529, 187903, 187904, 187905, 189554, 189555, 189563, 189561, 189562, 189564, 189499, 189500, 189557, 189558, 189592, 189583, 189582, 189581, 189580, 189482, 189481, 189480, 189479, 189464, 189465, 187190, 187187, 187186, 187015, 187014, 132908, 132907, 133156, 132906, 133159, 133155, 133163, 133162, 133172, 133153, 133098, 133093, 133096, 133104, 133122, 133123, 133134, 133119, 133120, 133136, 133137, 133272, 133271, 133267, 133268, 139537, 139538, 249879, 249880, 249881, 249882, 249878, 249876, 249875, 249919, 249904, 249905, 250068, 249987, 249988, 238157, 238158, 249748, 249744, 249730, 250275, 250276, 250309, 249729, 238159, 238160, 250278, 250283, 250281, 250279, 250211, 250214, 250215, 250246, 250245, 250244, 250241, 250239, 250243, 238651, 250247, 250368, 250359, 250358, 250227, 242484, 242482, 242485, 242486, 242492, 236833, 236834, 242478, 242474, 242475, 242509, 242510, 242454, 242452, 242450, 242457, 242